# Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default

The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users. For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value that can be easily detected:


In [1]:
import pandas as pd
import numpy as np
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [2]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [3]:
#none value is also treated as nan
string_data[0]=None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

In pandas, we’ve adopted a convention used in the R programming language by referring to missing data as NA, which stands for not available. In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example).

 When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data. 

There is work ongoing in the pandas project to improve the internal details of how missing data is handled, but the user API functions, like pandas.isnull, abstract away many of the annoying details

In [4]:
##----NA handling methods---##
#1) dropna() 
#DataFrameName.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
#Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate. 
#axis: axis takes int or string value for rows/columns. Input can be 0 or 1 for Integer and ‘index’ or ‘columns’ for String.
#how: how takes string value of two kinds only (‘any’ or ‘all’). ‘any’ drops the row/column if ANY value is Null and ‘all’ drops only if ALL values are null.
#thresh: thresh takes integer value which tells minimum amount of na values to drop.
#subset: It’s an array which limits the dropping process to passed rows/columns through list.
#inplace: It is a boolean which makes the changes in data frame itself if True.

#2) fillna()- Fill in missing data with some value or using an interpolation method such as 'ffill' or 'bfill'.
#3) isnull()- Return boolean values indicating which values are missing/NA. 
#4) notnull()-Negation of isnull.



In [ ]:
#|fillna
#|DataFrame.fillna(value=None, method=None, axis=None, inplace=False, limit=None, downcast=None, **kwargs)
#|value : Static, dictionary, array, series or dataframe to fill instead of NaN.
#|method : Method is used if user doesn’t pass any value. Pandas has different methods like bfill, backfill or ffill which fills the place with value in the Forward index or Previous/Back respectively.
#|axis: axis takes int or string value for rows/columns. Input can be 0 or 1 for Integer and ‘index’ or ‘columns’ for String
#|inplace: It is a boolean which makes the changes in data frame itself if True.
#|limit : This is an integer value which specifies maximum number of consequetive forward/backward NaN value fills.
#|downcast : It takes a dict which specifies what dtype to downcast to which one. Like Float64 to int64.
#|**kwargs : Any other Keyword arguments

# Filing Out Missing Data

In [5]:
#
from numpy import nan as NA
data=pd.Series([1,NA,3.5,NA,7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [6]:
#is equivalent to:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [14]:
#
data = pd.DataFrame([[1., 6.5, NA], [1., NA, NA], 
                     [NA, NA, NA], [NA, 6.5, NA]], 
                    columns = list('abc'))
print(data)
print()
cleaned=data.dropna(axis=1,how='all') #drop when all nan on y axis(column axis)
print(cleaned)
print()
print(data.dropna(axis=1))

     a    b   c
0  1.0  6.5 NaN
1  1.0  NaN NaN
2  NaN  NaN NaN
3  NaN  6.5 NaN

     a    b
0  1.0  6.5
1  1.0  NaN
2  NaN  NaN
3  NaN  6.5

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3]


In [15]:
data

,a,b,c
0,1.0,6.5,NaN
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,NaN


In [19]:
data[4]=NA

In [20]:
data

,a,b,c,4
0,1.0,6.5,NaN,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,NaN,NaN


In [21]:
data.dropna(how='all') #axis=0 by default and how=all means when all nan present drop that row

,a,b,c,4
0,1.0,6.5,NaN,NaN
1,1.0,NaN,NaN,NaN
3,NaN,6.5,NaN,NaN



A related way to filter out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the thresh argument:


In [22]:
#
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:3,1]=np.nan
df.iloc[:2,2]=np.nan
df

,0,1,2
0,0.257453,NaN,NaN
1,0.359617,NaN,NaN
2,1.380493,NaN,0.750709
3,-0.348437,-1.060979,1.070314
4,-0.392197,-0.985743,0.127919
5,0.416276,-0.152820,-2.340349
6,-0.137656,0.125492,1.168669


In [23]:
df.dropna()

,0,1,2
3,-0.348437,-1.060979,1.070314
4,-0.392197,-0.985743,0.127919
5,0.416276,-0.152820,-2.340349
6,-0.137656,0.125492,1.168669


In [26]:
df.dropna(thresh=2) #tells max number of na to be dropped

,0,1,2
2,1.380493,NaN,0.750709
3,-0.348437,-1.060979,1.070314
4,-0.392197,-0.985743,0.127919
5,0.416276,-0.152820,-2.340349
6,-0.137656,0.125492,1.168669


# Filling In Missing Data

In [27]:
# 
df


,0,1,2
0,0.257453,NaN,NaN
1,0.359617,NaN,NaN
2,1.380493,NaN,0.750709
3,-0.348437,-1.060979,1.070314
4,-0.392197,-0.985743,0.127919
5,0.416276,-0.152820,-2.340349
6,-0.137656,0.125492,1.168669


In [30]:
df.fillna(0)

,0,1,2
0,0.257453,0.000000,0.000000
1,0.359617,0.000000,0.000000
2,1.380493,0.000000,0.750709
3,-0.348437,-1.060979,1.070314
4,-0.392197,-0.985743,0.127919
5,0.416276,-0.152820,-2.340349
6,-0.137656,0.125492,1.168669


In [31]:
df.fillna({1:0.5,2:1})

,0,1,2
0,0.257453,0.500000,1.000000
1,0.359617,0.500000,1.000000
2,1.380493,0.500000,0.750709
3,-0.348437,-1.060979,1.070314
4,-0.392197,-0.985743,0.127919
5,0.416276,-0.152820,-2.340349
6,-0.137656,0.125492,1.168669


In [34]:
#fillna returns a new object, but you can modify the existing object in-place:
df.fillna(0,inplace=True)
df # now this original is modified by inplace=True parameter 

,0,1,2
0,0.257453,0.000000,0.000000
1,0.359617,0.000000,0.000000
2,1.380493,0.000000,0.750709
3,-0.348437,-1.060979,1.070314
4,-0.392197,-0.985743,0.127919
5,0.416276,-0.152820,-2.340349
6,-0.137656,0.125492,1.168669


In [36]:
#
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:,2]=NA
df

,0,1,2
0,0.750946,-1.070668,-1.707764
1,0.621771,1.713425,-1.085658
2,-1.131551,NaN,-0.309570
3,0.052589,NaN,1.577662
4,-0.750032,NaN,NaN
5,0.661416,NaN,NaN


In [37]:
df.fillna(method='ffill')

,0,1,2
0,0.750946,-1.070668,-1.707764
1,0.621771,1.713425,-1.085658
2,-1.131551,1.713425,-0.309570
3,0.052589,1.713425,1.577662
4,-0.750032,1.713425,1.577662
5,0.661416,1.713425,1.577662


In [38]:
df.fillna(axis=1,method='ffill')

,0,1,2
0,0.750946,-1.070668,-1.707764
1,0.621771,1.713425,-1.085658
2,-1.131551,-1.131551,-0.309570
3,0.052589,0.052589,1.577662
4,-0.750032,-0.750032,-0.750032
5,0.661416,0.661416,0.661416


In [39]:
df

,0,1,2
0,0.750946,-1.070668,-1.707764
1,0.621771,1.713425,-1.085658
2,-1.131551,NaN,-0.309570
3,0.052589,NaN,1.577662
4,-0.750032,NaN,NaN
5,0.661416,NaN,NaN


In [40]:
df.fillna(method='ffill',limit = 2)

,0,1,2
0,0.750946,-1.070668,-1.707764
1,0.621771,1.713425,-1.085658
2,-1.131551,1.713425,-0.309570
3,0.052589,1.713425,1.577662
4,-0.750032,NaN,1.577662
5,0.661416,NaN,1.577662


In [43]:
#With fillna you can do lots of other things with a little creativity. For example, you might pass the mean or median value of a Series:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

# 7.2 Data Transformation
#   Removing Duplicates